In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cosmetics/Cosmetics.csv


In [2]:
%matplotlib inline

from pathlib import Path

import heapq
from collections import defaultdict

import matplotlib.pylab as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split

In [3]:
cos = pd.read_csv('../input/cosmetics/Cosmetics.csv')
cos.set_index('Trans. ', inplace = True)

In [4]:
cos.head()

,Bag,Blush,Nail Polish,Brushes,Concealer,Eyebrow Pencils,Bronzer,Lip liner,Mascara,Eye shadow,Foundation,Lip Gloss,Lipstick,Eyeliner
Trans.,,,,,,,,,,,,,,
1,0,1,1,1,1,0,1,1,1,0,0,0,0,1
2,0,0,1,0,1,0,1,1,0,0,1,1,0,0
3,0,1,0,0,1,1,1,1,1,1,1,1,1,0
4,0,0,1,1,1,0,1,0,0,0,1,0,0,1
5,0,1,0,0,1,0,1,1,1,1,0,1,1,0


In [5]:
cos.columns

Index(['Bag', 'Blush', 'Nail Polish', 'Brushes', 'Concealer',
       'Eyebrow Pencils', 'Bronzer', 'Lip liner', 'Mascara', 'Eye shadow',
       'Foundation', 'Lip Gloss', 'Lipstick', 'Eyeliner'],
      dtype='object')

In [6]:
len(cos)

1000

In [7]:
#create frequent itemsets
itemsets = apriori(cos, min_support = 0.1, use_colnames = True)

#and convert into rules
rules = association_rules(itemsets, metric = 'confidence', min_threshold = 0.5)
rules.sort_values(by = ['lift'], ascending = False).head(12)

print(rules.sort_values(by = ['lift'], ascending = False)
     .drop(columns = ['antecedent support', 'consequent support', 'conviction'])
     .head(12))

                  antecedents               consequents  support  confidence  \
6                   (Brushes)             (Nail Polish)    0.149    1.000000   
7               (Nail Polish)                 (Brushes)    0.149    0.532143   
130   (Eye shadow, Concealer)       (Mascara, Eyeliner)    0.114    0.567164   
127       (Mascara, Eyeliner)   (Eye shadow, Concealer)    0.114    0.651429   
119          (Mascara, Blush)   (Eye shadow, Concealer)    0.119    0.646739   
120   (Eye shadow, Concealer)          (Mascara, Blush)    0.119    0.592040   
121       (Blush, Eye shadow)      (Mascara, Concealer)    0.119    0.653846   
118      (Mascara, Concealer)       (Blush, Eye shadow)    0.119    0.583333   
129    (Eye shadow, Eyeliner)      (Mascara, Concealer)    0.114    0.626374   
128      (Mascara, Concealer)    (Eye shadow, Eyeliner)    0.114    0.558824   
137      (Lip Gloss, Mascara)  (Foundation, Eye shadow)    0.111    0.613260   
136  (Foundation, Eye shadow)      (Lip 

If Brushes are purchased, then with confidence 100% Nail Polish will also be purchased. This rule has a lift ratio of 3.57.
If Nail Polish is purchased, then with confidence 53% Brushed will also be purchased. This rule has a lift ratio of 3.57.
If Eyeliner and Mascara are purchased, then with confidence 65% Eye shadow and Concealer will also be purchased. This rule has a lift ratio of 0.08

The redundant antecedents are rules 127&119; 122&129
Do a resaerch on how to solve redundancy issues and how to access their utilities